## Análise de Segmentação de Clientes: Clustering

In [60]:
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

df = pd.read_excel(url)

#### Análise exploratória (EDA)

##### Visualizando as primeiras linhas dos dados

In [61]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


##### Atributos do conjunto de dados

In [62]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

##### Tamanho do conjunto de dados

In [63]:
df.shape

(541909, 8)

##### Verificando os tipos de dados das colunas

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


##### Verificar se existem dados faltantes

In [65]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

##### Tratando os dados faltantes

In [66]:
#Adicinando a frase 'sem descrição' nos campos onde a descrição está faltando
df['Description'] = df['Description'].fillna('Sem descrição')

#Como tem clientes sem ID, vamos removê-los para não atrapalhar a segmentação
df.dropna(subset=['CustomerID'], inplace=True)

##### Verificando novamente a existência de dados faltantes

In [67]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

#### Calculando métricas RFM:

- Recência (R): Tempo desde a última compra do cliente.
- Frequência (F): Número total de compras feitas pelo cliente.
- Valor Monetário (M): Total gasto por cada cliente.

##### Recência (R)

In [68]:
# Definir a data de referência (última data no conjunto de dados)
latest_date = df['InvoiceDate'].max()

# Calcular a recência de cada cliente (em dias)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days  # Última compra de cada cliente
})
rfm.rename(columns={'InvoiceDate': 'Recency'}, inplace=True)

In [69]:
rfm.head()

,Recency
CustomerID,
12346.0,325
12347.0,1
12348.0,74
12349.0,18
12350.0,309


##### Frequência (F)

In [70]:
# Calcular a frequência de compras de cada cliente
rfm['Frequency'] = df.groupby('CustomerID').size()

In [71]:
rfm.head()

,Recency,Frequency
CustomerID,,
12346.0,325,2
12347.0,1,182
12348.0,74,31
12349.0,18,73
12350.0,309,17


##### Valor Monetário (M)

In [73]:
# Calcular o valor gasto por cada cliente (Quantity * UnitPrice)
df['TotalSpend'] = df['Quantity'] * df['UnitPrice']

# Calcular o valor monetário total gasto por cliente
rfm['Monetary'] = df.groupby('CustomerID')['TotalSpend'].sum()

In [30]:
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,325,2,0.00
12347.0,1,182,4310.00
12348.0,74,31,1797.24
12349.0,18,73,1757.55
12350.0,309,17,334.40


##### Relizando a segmentação de clientes

In [74]:
import pandas as pd
import numpy as np

# Exemplo de DataFrame com dados de e-commerce (substitua pelo seu DataFrame real)
# df = pd.read_csv('caminho/para/o/seu/arquivo.csv')

# Tratamento dos dados (geralmente aqui você já faz a limpeza e formatação)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Definindo a data de referência para calcular a Recência
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# Calculando RFM
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recência: dias desde a última compra
    'InvoiceNo': 'count',  # Frequência: número de compras
    'TotalPrice': 'sum'  # Valor Monetário: total gasto
})

rfm.rename(columns={'InvoiceDate': 'Recencia', 'InvoiceNo': 'Frequencia', 'TotalPrice': 'Valor'}, inplace=True)

# Atribuindo Scores para Recência, Frequência e Valor Monetário com base em quantis (5 quantis)
rfm['Recencia_Score'] = pd.qcut(rfm['Recencia'], 5, labels=[5, 4, 3, 2, 1])
rfm['Frequencia_Score'] = pd.qcut(rfm['Frequencia'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm['Valor_Score'] = pd.qcut(rfm['Valor'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

# Calculando o Score Total (Soma dos Scores de Recência, Frequência e Valor)
rfm['Total_Score'] = rfm['Recencia_Score'].astype(int) + rfm['Frequencia_Score'].astype(int) + rfm['Valor_Score'].astype(int)

# Segmentação com base no Total_Score
def segmentacao(row):
    if row['Total_Score'] >= 13:
        return 'Segmento 1: Clientes mais valiosos'
    elif row['Total_Score'] >= 9:
        return 'Segmento 2: Clientes potenciais'
    elif row['Total_Score'] >= 5:
        return 'Segmento 3: Clientes regulares'
    else:
        return 'Segmento 4: Clientes inativos'

rfm['Segmento'] = rfm.apply(segmentacao, axis=1)

# Exibindo o resultado final
rfm

,Recencia,Frequencia,Valor,Recencia_Score,Frequencia_Score,Valor_Score,Total_Score,Segmento
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,3,Segmento 4: Clientes inativos
12347.0,2,182,4310.00,5,5,5,15,Segmento 1: Clientes mais valiosos
12348.0,75,31,1797.24,2,3,4,9,Segmento 2: Clientes potenciais
12349.0,19,73,1757.55,4,4,4,12,Segmento 2: Clientes potenciais
12350.0,310,17,334.40,1,2,2,5,Segmento 3: Clientes regulares
...,...,...,...,...,...,...,...,...
18280.0,278,10,180.60,1,1,1,3,Segmento 4: Clientes inativos
18281.0,181,7,80.82,1,1,1,3,Segmento 4: Clientes inativos
18282.0,8,13,176.60,5,1,1,7,Segmento 3: Clientes regulares
